<a href="https://colab.research.google.com/github/Kushagra2000/Neural_machine_translation/blob/main/Machine_translation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
#imports
%tensorflow_version 1.x
print(tf.__version__)
import tensorflow as tf
from tensorflow import keras
import pandas as pd
import numpy as np
import re
import unicodedata
from sklearn.model_selection import train_test_split
from tensorflow.keras.layers import LSTM, Dense, TimeDistributed, Activation,InputLayer,Embedding
from tensorflow.keras.losses import sparse_categorical_crossentropy
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
# !pip install -q -U keras-tuner
# import kerastuner as kt 
import IPython

tf.test.gpu_device_name()

1.15.2


'/device:GPU:0'

In [3]:
#Upload the deu.txt file and put it in the Data Folder
#!ls Data/

In [4]:
#reading lines from the txt file
with open('drive/MyDrive/deu.txt','r',encoding='utf-8') as f:
  lines=(f.readlines())


In [5]:
#Hyperparameters go here
num_examples=15000
test_size=0.3
m1_lr=0.001
m2_lr=0.005
m1_lstm_units=64
m2_lstm_units=64
m1_epochs=25
m2_epochs=30
m1_batch_size=128
m2_batch_size=128
m2_embedding_col=64

In [7]:
#removing the unnecessary data and obtaining sentence pairs 
def process_text(lines,translate_mode):
  ''' 
  Takes a list of tab seperated
  English and German sentences as input 
  and returns dataframe of processed
  English and German sentence pairs
  '''
  proc_lines=[]
  if translate_mode==False:
    for line in lines:
      line=line.strip() #to remove newlines at the end of a sentence
      line=line.split("\t") #splitting by tabs
      line=line[:-1]  #remove contributing information
      
      line[0]=''.join(c for c in unicodedata.normalize('NFD', line[0]) if unicodedata.category(c) != 'Mn')
      line[0]=line[0].encode('utf8','ignore').decode('utf8')
      line[0]=line[0].replace(u'\u200b',' ')
      line[0]=line[0].lower()
      line[0]=line[0].replace('\xa0', ' ')
      line[0]=re.sub(r"([.,?!;])",r" \1 ",line[0])   #adding spaces before and after punctuation
      line[0]=re.sub(r"[0-9]"," ",line[0])  #removing numbers
      line[0]=re.sub(r'["]'," ",line[0])  #removing quotes
      line[0]=re.sub(r"[']","",line[0])
      line[0]=re.sub(r"[%-,]"," ",line[0])
      line[0]=re.sub(r"[:]"," ",line[0])
      line[0]=re.sub(r'[" "]+'," ",line[0])  #removing excess spaces
      line[0]=line[0].strip() #removing spaces from the end of string
      line[0]="<SOS> "+line[0]+" <EOS>"

      line[1]=''.join(c for c in unicodedata.normalize('NFD', line[1]) if unicodedata.category(c) != 'Mn')      
      line[1]=line[1].replace(u'\u200b',' ')
      line[1]=line[1].replace('\xa0', ' ')
      line[1]=line[1].lower()
      line[1]=re.sub(r"([.,?!;])",r" \1 ",line[1])
      line[1]=re.sub(r"[0-9]",r" ",line[1])
      line[1]=re.sub(r'["]',"",line[1])
      line[1]=re.sub(r'[—]',"",line[1])
      line[1]=re.sub(r'[„]',"",line[1])
      line[1]=re.sub(r'[“]',"",line[1])
      line[1]=re.sub(r'[–]',"",line[1])
      line[1]=re.sub(r'[‘‚]',"",line[1])
      line[1]=re.sub(r"[']","",line[1])
      line[1]=re.sub(r"[%()-,]"," ",line[1])
      line[1]=re.sub(r"[:]"," ",line[1])
      line[1]=re.sub(r'[" "]+'," ",line[1])
      line[1]=line[1].strip()
      line[1]="<SOS> "+line[1]+" <EOS>"

      proc_lines.append(line) 
    return pd.DataFrame.from_records(data=proc_lines,columns=['eng','ger'])
  else:
    lines=lines.strip()
    lines=''.join(c for c in unicodedata.normalize('NFD', lines) if unicodedata.category(c) != 'Mn')
    lines=lines.encode('utf8','ignore').decode('utf8')
    lines=lines.replace(u'\u200b',' ')
    lines=lines.lower()
    lines=lines.replace('\xa0', ' ')
    lines=re.sub(r"([.,?!;])",r" \1 ",lines)   #adding spaces before and after punctuation
    lines=re.sub(r"[0-9]"," ",lines)
    lines=re.sub(r'["]'," ",lines)
    lines=re.sub(r"[']","",lines)
    lines=re.sub(r"[%-,]"," ",lines)
    lines=re.sub(r"[:]"," ",lines)
    lines=re.sub(r'[" "]+'," ",lines)  #removing excess spaces
    lines=lines.strip() #removing spaces from the end of string
    lines="<SOS> "+lines+" <EOS>"
    return lines

  
     
  

In [8]:
#reading in the dataset, sampling randomly, shuffling and removing duplicates 
df=process_text(lines,False)
df=df.sample(n=num_examples, random_state=1)
df.drop_duplicates(subset="eng",inplace=True)
df = df.sample(frac=1).reset_index(drop=True)
df.head(10)

,eng,ger
0,<SOS> tom wont let mary drive . <EOS>,<SOS> tom lasst maria nicht ans steuer . <EOS>
1,<SOS> do you like doing that ? <EOS>,<SOS> macht ihr das gerne ? <EOS>
2,<SOS> tom and mary are creative . <EOS>,<SOS> tom und maria sind kreativ . <EOS>
3,<SOS> youll need money . <EOS>,<SOS> sie werden geld brauchen . <EOS>
4,<SOS> the firemen battered down the door . <EOS>,<SOS> die feuerwehrmanner schlugen die tur ein...
5,<SOS> this dam supplies us with water and elec...,<SOS> dieser damm liefert uns wasser und strom...
6,<SOS> tom and mary are jealous . <EOS>,<SOS> tom und maria sind eifersuchtig . <EOS>
7,<SOS> i actually did that yesterday . <EOS>,<SOS> das habe ich gestern schon gemacht . <EOS>
8,<SOS> this soup is too thin for me . <EOS>,<SOS> diese suppe ist mir zu dunn . <EOS>
9,<SOS> he turned on the radio . <EOS>,<SOS> er schaltete das radio ein . <EOS>


In [9]:
df.tail(10)

,eng,ger
14706,<SOS> i didnt mean to make tom cry . <EOS>,<SOS> ich wollte tom nicht zum weinen bringen ...
14707,<SOS> i want to know why youre here . <EOS>,<SOS> ich mochte wissen warum du hier bist . <...
14708,<SOS> you are as tall as i am . <EOS>,<SOS> sie sind so groß wie ich es bin . <EOS>
14709,<SOS> we have something in common . <EOS>,<SOS> wir haben eine gemeinsamkeit . <EOS>
14710,<SOS> were a little older . <EOS>,<SOS> wir sind ein bisschen alter . <EOS>
14711,<SOS> we happened to get on the same bus . <EOS>,<SOS> wir sind zufallig in denselben bus einge...
14712,<SOS> when was the last time you watched a mov...,<SOS> wann hast du dir das letzte mal einen fi...
14713,<SOS> ill bring it to you tomorrow . <EOS>,<SOS> ich bringe es euch morgen . <EOS>
14714,<SOS> whats happened to your eye ? <EOS>,<SOS> was ist mit deinem auge passiert ? <EOS>
14715,<SOS> did tom say why ? <EOS>,<SOS> hat tom gesagt warum ? <EOS>


In [10]:
class Vocab_builder():
  '''
  Builds vocabulary and 
  word to index and index to word dictionaries
  from dataset
  '''
  def __init__(self,lang,series):
    self.lang=lang
    self.data=series
  def tokenize(self,line):
    return line.split(' ')
  def build_vocab(self):
    self.uniq_words=set()
    
    self.maxlen=0
    count=3
    self.num_list=[]
    for index,line in self.data.items():
      self.word_list=self.tokenize(line)
      self.maxlen=max(len(self.word_list),self.maxlen)
      for word in self.word_list:
        if(word not in self.uniq_words and word!='<EOS>' and word!='<SOS>'):
          self.uniq_words.add(word)
          self.num_list.append(count)
          count+=1
      
    self.vocab_list=['<PAD>','<SOS>','<EOS>']+sorted(list(self.uniq_words))
    self.num_list=[0,1,2]+self.num_list
    print("Built vocabulary having {} elements".format(len(self.vocab_list)))
    print("Largest sentence length (with tags):{}".format(self.maxlen))
    return dict(zip(self.vocab_list,self.num_list)),dict(zip(self.num_list,self.vocab_list))






In [11]:
#Objects of Vocab_builder class
eng=Vocab_builder('eng',df['eng'])
ger=Vocab_builder('ger',df['ger'])


In [12]:
#English word to index and index to word dictionaries
eng_w2i,eng_i2w=eng.build_vocab()


Built vocabulary having 5903 elements
Largest sentence length (with tags):31


In [13]:
#checking for special characters in English dictionary
for i in sorted (eng_w2i.keys())[:20] :  
     print(i, end = " ") 

! $ -to- . ; <EOS> <PAD> <SOS> ? a abandon abandoned abducted ability able aboard about above abroad absent 

In [14]:
#German word to index and index to word dictionaries
ger_w2i,ger_i2w=ger.build_vocab()

Built vocabulary having 9004 elements
Largest sentence length (with tags):30


In [15]:
#checking if the index dictionaries are correct
'geh'==ger_i2w[ger_w2i['geh']]

True

In [16]:
'go'==eng_i2w[eng_w2i['go']]

True

In [17]:
#checking for special characters in German dictionary
for i in sorted (ger_w2i.keys())[:20] :  
     print(i, end = " ") 

! $ - . ; <EOS> <PAD> <SOS> ? ab abbekommen abbezahlen abbiegen abbringen abend abendbrot abende abendessen abendkleid abendkleider 

In [18]:
def sent_to_ind(sentence,lang):
  '''
  Tokenizes a string and
  converts it to an np array of 
  indices and pads the 
  array according to max sentence length
  '''
  ind_list=[]
  if lang=='eng':
    tokens=eng.tokenize(sentence)
    for token in tokens:
      ind_list.append(eng_w2i[token])
    while len(ind_list)<max(ger.maxlen,eng.maxlen):
      ind_list.append(0)
  else:
    tokens=ger.tokenize(sentence)
    for token in tokens:
      ind_list.append(ger_w2i[token])
    while len(ind_list)<max(ger.maxlen,eng.maxlen):
      ind_list.append(0)
    
  return np.array(ind_list)



In [19]:
#Checking correctness of sentence to index conversion
test_lis=sent_to_ind('<SOS> tom wird bald hier sein . was heißt bald ? <EOS>','ger')

print(len(test_lis))
print(test_lis)

31
[   1 7289 8594  760 3778 6624    6 8352 3693  760    8    2    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0]


In [20]:
test_lis=sent_to_ind("<SOS> tom will be here soon . how soon ? <EOS>",'eng')
print(len(test_lis))
print(test_lis)

31
[   1 5312 5770  457 2477 4788    6 2570 4788    8    2    0    0    0
    0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0    0]


In [21]:
#splitting the data into training,testing and validation sets

train_x,test_x,train_y,test_y=train_test_split(df['eng'],df['ger'],test_size=0.1,random_state=42)
train_x,val_x,train_y,val_y=train_test_split(train_x,train_y,test_size=0.23,random_state=42)


In [22]:
def sent_to_np(series,lang,translate_mode):
  '''
  Converts a dataframe column to 
  a unsqueezed np array of indexes
  with padding for feeding into NN
  '''
  ret_list=[]
  if translate_mode==False :
    if lang=='eng':
      for index,val in series.items():
        ret_list.append(sent_to_ind(val,'eng'))
    else:
      for index,val in series.items():
        ret_list.append(sent_to_ind(val,'ger'))
    
    ret_list=np.array(ret_list)
    return np.expand_dims(ret_list,axis=2)
  else:
    ans=sent_to_ind(series,'eng')
    ans=np.expand_dims(ans,axis=0)
    ans=np.expand_dims(ans,axis=2)
    return ans


In [23]:
train_x=sent_to_np(train_x,'eng',False)
train_y=sent_to_np(train_y,'ger',False)
test_x=sent_to_np(test_x,'eng',False)
test_y=sent_to_np(test_y,'ger',False)
val_x=sent_to_np(val_x,'eng',False)
val_y=sent_to_np(val_y,'ger',False)


In [24]:
#loading training,testing and validation data

# train_x=np.load("drive/MyDrive/data/training/train_x.npy")
# train_y=np.load("drive/MyDrive/data/training/train_y.npy")

# test_x=np.load("drive/MyDrive/data/testing/test_x.npy")
# test_y=np.load("drive/MyDrive/data/testing/test_y.npy")

# val_x=np.load("drive/MyDrive/data/validation/val_x.npy")
# val_y=np.load("drive/MyDrive/data/validation/val_y.npy")



In [25]:
# [batch,timesteps,feature]
print(train_x.shape)
print(train_y.shape)

(10197, 31, 1)
(10197, 31, 1)


In [26]:
print(test_x.shape)
print(test_y.shape)

(1472, 31, 1)
(1472, 31, 1)


In [27]:
print(val_x.shape)
print(val_y.shape)

(3047, 31, 1)
(3047, 31, 1)


In [36]:
def translate(sentence,mod,embedded):
  '''
  Function for translating given English sentence
  to German using model predictions
  '''
  ans=""
  preproc_sent=process_text(sentence,True)
  #print(preproc_sent)
  model_inp=sent_to_np(preproc_sent,'eng',True)
  if (embedded):
    model_inp=np.squeeze(model_inp,axis=2)
  #print(model_inp)
  pred=mod.predict(model_inp)
  #print(pred)
  for i in pred[0]:
    ind=np.argmax(i)
    #print(ind)
    #print("MAX:",i[ind])
    #print(i[24])
    if(ger_i2w[ind]=='<SOS>' or ger_i2w[ind]=='<EOS>' or ger_i2w[ind]=='<PAD>'):
        continue
    ans+=ger_i2w[ind]
    ans+=" "
  return ans

In [ ]:
#training starts here ----------------------------------------------------------

In [28]:

#Removing singleton axis  3rd axis
#for embedding layer
t_x=np.squeeze(train_x,axis=2)
v_x=np.squeeze(val_x,axis=2)
te_x=np.squeeze(test_x,axis=2)
print(t_x.shape)
print(v_x.shape)
print(te_x.shape)

(10197, 31)
(3047, 31)
(1472, 31)


In [ ]:
# def f1_score(y_true,y_pred):
#   p=tf.keras.metrics.Precision()
#   r=tf.keras.metrics.Recall()
#   p.update_state(y_true,y_pred)
#   r.update_state(y_true,y_pred)
#   return (2*p.result()*r.result())/(p.result()+r.result());
# def leaky_relu(x):
#     return tf.nn.leaky_relu(x, alpha=0.3)


def base_LSTM_model_HP_TUNING(hp):
  '''
  Simple LSTM model for hyperparameter tuning
  '''
  m1_lstm_units=hp.Int('units',min_value=32,max_value=128,step=32)
  m1_lr=hp.Choice('learning_rate', values = [0.001,0.003,0.005,0.008,0.01,0.05]) 
  lstm=LSTM(m1_lstm_units,return_sequences=True,activation='tanh')  #LSTM layer with output being hiddent state at time t
  layer_at_t=TimeDistributed(Dense(len(ger.vocab_list),activation='softmax')) #Dense layer acting on hidden output at each step to generate predictions
  model=tf.keras.Sequential()
  model.add(InputLayer(train_x.shape[1:]))
  model.add(lstm)
  model.add(layer_at_t)

  model.compile(loss=sparse_categorical_crossentropy,optimizer=Adam(m1_lr),metrics=['accuracy'])
  return model


In [43]:



def embedding_LSTM_HP_TUNING(hp):
  '''
  LSTM model with embedding layer for hyperparameter tuning
  '''
  m2_lstm_units=hp.Int('units',min_value=32,max_value=128,step=32)
  m2_lr=hp.Choice('learning_rate', values = [0.001,0.003,0.005,0.008,0.01,0.05,0.08])
  embedding_col=hp.Int('output_dim',min_value=32,max_value=160,step=32)
  lstm=LSTM(m2_lstm_units,return_sequences=True,activation='tanh')
  print(t_x.shape[1])
  embed=Embedding(len(ger.vocab_list),embedding_col,input_length=t_x.shape[1])
  layer_at_t=TimeDistributed(Dense(len(ger.vocab_list),activation="softmax"))
  model=tf.keras.Sequential()
  model.add(embed)
  model.add(lstm)
  model.add(layer_at_t)

  model.compile(loss=sparse_categorical_crossentropy,optimizer=Adam(m2_lr),metrics=['accuracy'])
  return model
  

In [44]:
class ClearTrainingOutput(tf.keras.callbacks.Callback):
  def on_train_end(*args, **kwargs):
    IPython.display.clear_output(wait = True)



In [ ]:
#Hyperparameter tuning starts here
#Hyperparameter tuning of base model
tuner = kt.BayesianOptimization(base_LSTM_model_HP_TUNING,
                     objective = 'val_accuracy', 
                     num_initial_points=50,
                     max_trials=15,
                     directory='./',
                     project_name='base_LSTM')


In [ ]:
tuner.search(train_x,train_y, epochs = 15, validation_data = (val_x, val_y), callbacks = [ClearTrainingOutput()])

Trial 16 Complete [00h 02m 53s]
val_accuracy: 0.7552497386932373

Best val_accuracy So Far: 0.765637218952179
Total elapsed time: 00h 47m 45s
INFO:tensorflow:Oracle triggered exit


In [ ]:
best_hps1 = tuner.get_best_hyperparameters(num_trials = 1)[0]

In [ ]:
best_hps1.get('units')

128

In [ ]:
best_hps1.get('learning_rate')

0.001

In [ ]:
tuner2 = kt.BayesianOptimization(embedding_LSTM_HP_TUNING,
                     objective = 'val_accuracy', 
                     num_initial_points=50,
                     max_trials=30,
                     directory='./',
                     project_name='embedding')


31


In [ ]:
tuner2.search(t_x,train_y, epochs = 30, validation_data = (v_x, val_y), callbacks = [ClearTrainingOutput()])


Trial 30 Complete [00h 07m 08s]
val_accuracy: 0.8056359887123108

Best val_accuracy So Far: 0.8067874908447266
Total elapsed time: 03h 46m 53s
INFO:tensorflow:Oracle triggered exit


In [ ]:
best_hps2 = tuner2.get_best_hyperparameters(num_trials = 1)[0]

In [ ]:
best_hps2.get('units')

128

In [ ]:
best_hps2.get('learning_rate')

0.01

In [ ]:
best_hps2.get('output_dim')

96

In [29]:
#Making base model using best hyperparameters
def base_LSTM_model(m1_lstm_units,m1_lr):
  '''
  Simple LSTM model
  '''
  lstm=LSTM(m1_lstm_units,return_sequences=True,activation='tanh')  #LSTM layer with output being hiddent state at time t
  layer_at_t=TimeDistributed(Dense(len(ger.vocab_list),activation='softmax')) #Dense layer acting on hidden output at each step to generate predictions
  model=tf.keras.Sequential()
  model.add(InputLayer(train_x.shape[1:]))
  model.add(lstm)
  model.add(layer_at_t)

  model.compile(loss=sparse_categorical_crossentropy,optimizer=Adam(m1_lr),metrics=['accuracy'])
  return model

In [30]:
#Making embedding model using hyperparameters
def embedding_LSTM(m2_lstm_units,m2_lr,embedding_col):
  '''
  LSTM model with embedding layer
  '''
  lstm=LSTM(m2_lstm_units,return_sequences=True,activation='tanh')
  print(t_x.shape[1])
  embed=Embedding(len(ger.vocab_list),embedding_col,input_length=t_x.shape[1])
  layer_at_t=TimeDistributed(Dense(len(ger.vocab_list),activation="softmax"))
  model=tf.keras.Sequential()
  model.add(embed)
  model.add(lstm)
  model.add(layer_at_t)

  model.compile(loss=sparse_categorical_crossentropy,optimizer=Adam(m2_lr),metrics=['accuracy'])
  return model

In [ ]:
base_model=base_LSTM_model(best_hps1.get('units'),best_hps1.get('learning_rate'))
base_model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 31, 128)           66560     
_________________________________________________________________
time_distributed_1 (TimeDist (None, 31, 10464)         1349856   
Total params: 1,416,416
Trainable params: 1,416,416
Non-trainable params: 0
_________________________________________________________________


In [ ]:
#training the base model

base_model.fit(train_x,train_y,m1_batch_size,m1_epochs,validation_data=(val_x,val_y))
#base_model.save("Less_trained_base.h5")

Epoch 1/25
106/106 [==============================] - 8s 69ms/step - loss: 5.8066 - accuracy: 0.6356 - val_loss: 1.9201 - val_accuracy: 0.7372
Epoch 2/25
106/106 [==============================] - 7s 65ms/step - loss: 1.8705 - accuracy: 0.7453 - val_loss: 1.8546 - val_accuracy: 0.7464
Epoch 3/25
106/106 [==============================] - 7s 66ms/step - loss: 1.8101 - accuracy: 0.7479 - val_loss: 1.7521 - val_accuracy: 0.7449
Epoch 4/25
106/106 [==============================] - 7s 65ms/step - loss: 1.6714 - accuracy: 0.7497 - val_loss: 1.5917 - val_accuracy: 0.7490
Epoch 5/25
106/106 [==============================] - 7s 66ms/step - loss: 1.5449 - accuracy: 0.7511 - val_loss: 1.5704 - val_accuracy: 0.7495
Epoch 6/25
106/106 [==============================] - 7s 66ms/step - loss: 1.5332 - accuracy: 0.7501 - val_loss: 1.5638 - val_accuracy: 0.7492
Epoch 7/25
106/106 [==============================] - 7s 66ms/step - loss: 1.5077 - accuracy: 0.7528 - val_loss: 1.5578 - val_accuracy: 0.7523

In [ ]:
# base_model=tf.keras.models.load_model("drive/MyDrive/model/Less_trained_base.h5")
# base_model.summary()

In [ ]:
#Evaluating LSTM model on unseen data

l,acc=base_model.evaluate(test_x,test_y)
print("Base model loss for testing set:{}".format(l))
print("Base model accuracy for testing set:{}".format(acc))


61/61 [==============================] - 1s 18ms/step - loss: 1.5160 - accuracy: 0.7668
Base model loss for testing set:1.5159815549850464
Base model accuracy for testing set:0.7668282985687256


In [ ]:
#Sample translation by base LSTM model
translate("I am sad",base_model,False)

'<SOS> ich und maria . <EOS> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> <PAD> '

In [ ]:
embeded_model=embedding_LSTM(best_hps2.get('units'),best_hps2.get('learning_rate'),best_hps2.get('output_dim'))
embeded_model.summary()

31
Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 31, 96)            1004544   
_________________________________________________________________
lstm_3 (LSTM)                (None, 31, 128)           115200    
_________________________________________________________________
time_distributed_3 (TimeDist (None, 31, 10464)         1349856   
Total params: 2,469,600
Trainable params: 2,469,600
Non-trainable params: 0
_________________________________________________________________


In [ ]:
#training embedding model
embeded_model.fit(t_x,train_y,batch_size=m2_batch_size,epochs=15,validation_data=(v_x,val_y))

Epoch 1/15
106/106 [==============================] - 10s 79ms/step - loss: 3.4675 - accuracy: 0.6852 - val_loss: 1.7224 - val_accuracy: 0.7650
Epoch 2/15
106/106 [==============================] - 8s 75ms/step - loss: 1.6299 - accuracy: 0.7704 - val_loss: 1.4325 - val_accuracy: 0.7758
Epoch 3/15
106/106 [==============================] - 8s 75ms/step - loss: 1.3367 - accuracy: 0.7851 - val_loss: 1.3438 - val_accuracy: 0.7895
Epoch 4/15
106/106 [==============================] - 8s 75ms/step - loss: 1.2197 - accuracy: 0.7976 - val_loss: 1.3081 - val_accuracy: 0.7939
Epoch 5/15
106/106 [==============================] - 8s 76ms/step - loss: 1.1024 - accuracy: 0.8090 - val_loss: 1.2848 - val_accuracy: 0.7987
Epoch 6/15
106/106 [==============================] - 8s 77ms/step - loss: 1.0089 - accuracy: 0.8162 - val_loss: 1.2728 - val_accuracy: 0.8010
Epoch 7/15
106/106 [==============================] - 8s 76ms/step - loss: 0.9093 - accuracy: 0.8264 - val_loss: 1.2701 - val_accuracy: 0.800

In [31]:
embeded_model=tf.keras.models.load_model("drive/MyDrive/embedded_15000.h5")
l,acc=embeded_model.evaluate(te_x,test_y)
print("Embedded model loss for testing set:{}".format(l))
print("Embedded model accuracy for testing set:{}".format(acc))


Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
1472/1472 [==============================] - 2s 1ms/sample - loss: 0.8028 - acc: 0.8684
Embedded model loss for testing set:0.8028346928565399
Embedded model accuracy for testing set:0.8683818578720093


In [37]:
translate("i am very sad ",embeded_model,True)

'ich bin sehr glucklich . '